## 1. Cargar datos

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import  pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# fix random seed for reproducibility
np.random.seed(7)



dataframe = pd.read_csv("datos.csv", header=None, delimiter = ';')
dataset = dataframe.values
dataset = dataset.astype('float32')

# normalize the dataset
#scaler = MinMaxScaler(feature_range=(0, 1))
#dataset = scaler.fit_transform(dataset)


# split into train and test sets
train_size = int(len(dataset) * 0.55)
test_size = int(len(dataset)*0.25)

train, test, val = dataset[0:train_size,:], dataset[train_size:(train_size+test_size),:], dataset[(train_size+test_size):len(dataset),:]


######

# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

from pandas import DataFrame
from pandas import concat
 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# u(t) es la primera columna de train
# y(t) es la segunda columna de train 

u_train = np.reshape(train[:,0],(-1,1))
u_test = np.reshape(test[:,0],(-1,1))
u_val =  np.reshape(val[:,0],(-1,1))
y_train = np.reshape(train[:,1],(-1,1))
y_test = np.reshape(test[:,1],(-1,1))
y_val = np.reshape(val[:,1],(-1,1))

plt.figure(figsize=(20,10))
plt.plot(u_train[0:500]) 
plt.plot(y_train[0:500])

plt.xlabel('k',fontsize = 20)
plt.ylabel('Magnitud normalizada',fontsize = 20)
plt.title('Entrada y salida serie de Chen',fontsize = 20)
plt.legend(('u(k)','y(k)'), loc = 'upper right')
plt.show()



u_min = np.amin(u_train)
u_max = np.amax(u_train)

y_min = np.amin(y_train)
y_max = np.amax(y_train)

#normalizar

u_train = (u_train-u_min)/(u_max-u_min)
y_train = (y_train-y_min)/(y_max-y_min)
u_test = (u_test-u_min)/(u_max-u_min)
u_val = (u_val-u_min)/(u_max-u_min)
y_test = (y_test-y_min)/(y_max-y_min)
y_val = (y_val-y_min)/(y_max-y_min)

Using Theano backend.


<Figure size 2000x1000 with 1 Axes>

In [3]:
# ENTRENAMIENTO
x_train = np.concatenate((u_train, y_train), axis=1) 
x_train_1 = np.array(series_to_supervised(x_train, n_in=3, n_out=1))[:,0:6]
y_train_1 = np.array(series_to_supervised(y_train, n_in=3, n_out=1))[:,1:4]
x_train_1 = np.reshape(x_train_1,(x_train_1.shape[0],3,2))
y_train_1 = np.reshape(y_train_1,((y_train_1.shape[0]),3,1))
y_train_triple = np.concatenate((y_train_1,y_train_1,y_train_1),axis=2)




# TEST 

x_test= np.concatenate((u_test, y_test), axis=1) 
x_test_1 = np.array(series_to_supervised(x_test, n_in=3, n_out=1))[:,0:6]
y_test_1 = np.array(series_to_supervised(y_test, n_in=3, n_out=1))[:,1:4]
x_test_1 = np.reshape(x_test_1,(x_test_1.shape[0],3,2))
y_test_1 = np.reshape(y_test_1,((y_test_1.shape[0]),3,1))
y_test_triple = np.concatenate((y_test_1,y_test_1,y_test_1),axis=2)


# VALIDACION

x_val= np.concatenate((u_val, y_val), axis=1) 
x_val_1 = np.array(series_to_supervised(x_val, n_in=3, n_out=1))[:,0:6]
y_val_1 = np.array(series_to_supervised(y_val, n_in=3, n_out=1))[:,1:4]
x_val_1 = np.reshape(x_val_1,(x_val_1.shape[0],3,2))
y_val_1 = np.reshape(y_val_1,((y_val_1.shape[0]),3,1))
y_val_triple = np.concatenate((y_val_1,y_val_1,y_val_1),axis=2)

In [4]:
def PINAW(intervalo_superior,intervalo_inferior,data):
    N = len(intervalo_superior)
    maximo = np.amax(data)
    minimo = np.amin(data)
    R = maximo - minimo
    op1 = np.subtract(intervalo_superior,intervalo_inferior) 
    op2 = (1/(N*R))*(np.sum(op1))
    return op2

def PICP(intervalo_superior,intervalo_inferior,data):
    N = len(intervalo_superior)
    contador = 0
    for i in range(0,len(data)):
        if intervalo_superior[i] > data[i] > intervalo_inferior[i]:
            contador +=1
    return contador/N 

In [5]:
## Calcular PINAW PICP
"""
Esta funcion permite calcular el valor del PINAW, PICP y RMSE de
las predicciones a n pasos.
x_val = datos [u(t),y(t)] con estos se forma la matriz de entrenamiento
y_max = maximo valor salida 
y_min = minimo valor de salida 
Estos son utilizados para desnormalizar las predicciones y calcular apropiadamente las metricas

z1 = Es el nodo de salida del grafo utilizado.
z1 contiene [y_lower,y_crisp,y_upper]



"""
def calcular_PINAW_PICP_RNN(n,x_val,y_max,y_min,z1):
    n_in = 3
    matriz_n_paso = np.array(series_to_supervised(x_val, n_in=3, n_out=n+1))
    matriz_n_paso = np.reshape(matriz_n_paso,((matriz_n_paso.shape[0]),n_in+(n+1),2))    
    for i in range(0,(n+1)):
        x = matriz_n_paso[:,0+i:n_in+i,:]  # [Muestras, Time steps, entradas]
        prediccion_val = z1.eval(feed_dict={X: x})
        y_lower = prediccion_val[:,-1,0] # devolver salidas ultimo time estep
        y_crisp = prediccion_val[:,-1,1]
        y_upper = prediccion_val[:,-1,2]        
        rmse = np.sqrt(mean_squared_error(y_crisp*(y_max-y_min)+y_min,
                                          matriz_n_paso[:,n_in+i,1]*(y_max-y_min)+y_min))
        A = PINAW(y_upper*(y_max-y_min)+y_min,y_lower*(y_max-y_min)+y_min,matriz_n_paso[:,n_in+i,1]*(y_max-y_min)+y_min)
        B = PICP(y_upper*(y_max-y_min)+y_min,y_lower*(y_max-y_min)+y_min,matriz_n_paso[:,n_in+i,1]*(y_max-y_min)+y_min)      
        matriz_n_paso[:,n_in+i,1]=y_crisp
        PINAW_percent = A*100
        PICP_percent = B*100
    return PINAW_percent,PICP_percent,rmse

In [6]:
def generar_mini_batch_rnn(X , y, batch_indice, batch_tamano):
    inicio = batch_indice*batch_tamano
    fin = (batch_indice+1)*batch_tamano
    X_batch = X[inicio:fin,:,:]
    y_batch = y[inicio:fin,:,:]
    return X_batch, y_batch

## 2. Grafo

In [7]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.layers import fully_connected

n_steps = 3
n_inputs = 2
n_neurons = 15
n_outputs = 3

def red_neuronal(parametro): 
    grafo = tf.Graph()    
    with grafo.as_default():
                
        # placeholders
        X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
        y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

        with tf.name_scope("RED_RECURRENTE") as scope:
            cell = tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, name ="celda")
            rnn_outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

            stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1,n_neurons])
            stacked_outputs = fully_connected(stacked_rnn_outputs, n_outputs,
                                             activation_fn=None)
            outputs = tf.reshape(stacked_outputs, [-1,n_steps,n_outputs])


        error = tf.subtract(y, outputs, name="error")

        with tf.name_scope("errores") as scope:
            error_lower = error[:,:,0]
            error_crisp = error[:,:,1]
            error_upper = error[:,:,2]

        with tf.name_scope("errores_medios"):
            mse_lower = tf.reduce_mean(tf.square(error_lower), name="mse_lower")
            mse_crisp = tf.reduce_mean(tf.square(error_crisp), name="mse_crisp")
            mse_upper = tf.reduce_mean(tf.square(error_upper), name="mse_upper")

            mse_upper1= tf.reduce_mean(tf.square(tf.maximum(error_upper, 0., name="relu1")))
            mse_lower1= tf.reduce_mean(tf.square(tf.maximum(-error_lower, 0., name="relu2")))

        with tf.name_scope('losses'):
            loss_upper = mse_upper + parametro*mse_upper1
            loss_lower = mse_lower + parametro*mse_lower1

            loss = mse_crisp + loss_upper + loss_lower

        learning_rate = 0.001
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        training_op = optimizer.minimize(loss)
        saver = tf.train.Saver()
        init = tf.global_variables_initializer()
    return grafo, X, y, outputs, init, saver, training_op, loss


Instructions for updating:
Use the retry module or similar alternatives.


## 3. Fase Ejecución

### 3.1 1 paso

In [10]:
batch_size = 100
n_batches = int(np.ceil(y_train_triple.shape[0] / batch_size))
n_epochs = 8000
loss_minimo = 1000

for lam in range(300,700,10):
        
        
    loss_minimo = 1000
    grafo, X, y, outputs, init, saver, training_op, loss = red_neuronal(lam)
    with tf.Session(graph=grafo) as sess:
        sess.run(init)

        for epoch in range(n_epochs):
            for batch_indice in range(n_batches):
                X_batch, y_batch = generar_mini_batch_rnn(x_train_1, y_train_triple, batch_indice, batch_size)
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            perdida = sess.run(loss, feed_dict={X: x_test_1, y: y_test_triple})
            if perdida < loss_minimo:
                loss_minimo = perdida
                contador = 0 # reiniciar contador
            else:
                contador +=1
                if contador == 6:
                    #print('El entrenamiento ha sido detenido')
                    break 

        PINAW_percent,PICP_percent,rmse = calcular_PINAW_PICP_RNN(1,x_test,y_max,y_min,outputs)
        print('Red con parametro:' ,lam,'PINAW test: ',PINAW_percent,'PICP test: ',PICP_percent,'RMSE test: ',rmse)
        if PICP_percent > 90.0:
                print(lam)
                
                parametro = lam
                #save_path = saver.save(sess, "./my_model_intervalo1.ckpt")
                print('Ya se ha logrado el requisito de PICP > 90')
                PINAW_percent_val,PICP_percent_val,rmse_val = calcular_PINAW_PICP_RNN(1,x_val,y_max,y_min,outputs)
                print('PINAW val: ',PINAW_percent_val,'PICP val: ',PICP_percent_val,'RMSE val: ',rmse_val)

                break



Red con parametro: 300 PINAW test:  14.5788340119 PICP test:  70.94188376753507 RMSE test:  0.644168
Red con parametro: 310 PINAW test:  13.0111594365 PICP test:  75.47094188376754 RMSE test:  0.568154
Red con parametro: 320 PINAW test:  14.293644537 PICP test:  73.30661322645291 RMSE test:  0.59464
Red con parametro: 330 PINAW test:  12.7009689741 PICP test:  66.37274549098197 RMSE test:  0.602293
Red con parametro: 340 PINAW test:  12.8831606828 PICP test:  67.97595190380761 RMSE test:  0.568639
Red con parametro: 350 PINAW test:  15.5117399598 PICP test:  71.30260521042085 RMSE test:  0.600251
Red con parametro: 360 PINAW test:  13.7906454584 PICP test:  62.44488977955912 RMSE test:  0.740155
Red con parametro: 370 PINAW test:  16.3183731711 PICP test:  78.59719438877755 RMSE test:  0.729166
Red con parametro: 380 PINAW test:  13.6105510508 PICP test:  71.18236472945891 RMSE test:  0.592807
Red con parametro: 390 PINAW test:  15.0086820191 PICP test:  74.9498997995992 RMSE test:  0.

In [11]:
best_PINAW = 100
for lam in range(0,100,1):
    loss_minimo = 1000
    grafo, X, y, outputs, init, saver, training_op, loss = red_neuronal(parametro)
    with tf.Session(graph=grafo) as sess:
        sess.run(init)
        
        for epoch in range(n_epochs):
            for batch_indice in range(n_batches):
                X_batch, y_batch = generar_mini_batch_rnn(x_train_1, y_train_triple, batch_indice, batch_size)
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            perdida = sess.run(loss, feed_dict={X: x_test_1, y: y_test_triple})
            
            if perdida < loss_minimo:
                loss_minimo = perdida
                contador = 0 # reiniciar contador
            else:
                contador +=1
                if contador == 6:
                    #print('El entrenamiento ha sido detenido')
                    break 

        PINAW_percent,PICP_percent,rmse = calcular_PINAW_PICP_RNN(1,x_test,y_max,y_min,outputs)
        print('Iteración:' ,lam,'PINAW test: ',PINAW_percent,'PICP test: ',PICP_percent,'RMSE test: ',rmse)
        if PICP_percent > 90.0 and PINAW_percent < best_PINAW:
                print(lam)
                best_PINAW = PINAW_percent
                save_path = saver.save(sess, "./mejor_modelo_rnn_LSTM_1paso.ckpt")
                print('Ya se ha logrado el requisito de PICP > 90')
                PINAW_percent_val,PICP_percent_val,rmse_val = calcular_PINAW_PICP_RNN(1,x_val,y_max,y_min,outputs)
                print('PINAW val: ',PINAW_percent_val,'PICP val: ',PICP_percent_val,'RMSE val: ',rmse_val)

                

Iteración: 0 PINAW test:  13.8914444996 PICP test:  70.54108216432866 RMSE test:  0.561992
Iteración: 1 PINAW test:  13.8752184892 PICP test:  66.61322645290582 RMSE test:  0.650144
Iteración: 2 PINAW test:  13.564458423 PICP test:  76.7935871743487 RMSE test:  0.574989
Iteración: 3 PINAW test:  13.5769797449 PICP test:  82.3246492985972 RMSE test:  0.494698
Iteración: 4 PINAW test:  16.2575387628 PICP test:  71.9438877755511 RMSE test:  0.621373
Iteración: 5 PINAW test:  16.5380518721 PICP test:  74.18837675350701 RMSE test:  0.737526
Iteración: 6 PINAW test:  16.4696505176 PICP test:  81.60320641282564 RMSE test:  0.710179
Iteración: 7 PINAW test:  14.2220619136 PICP test:  70.74148296593187 RMSE test:  0.62537
Iteración: 8 PINAW test:  14.0680524613 PICP test:  71.78356713426854 RMSE test:  0.615509
Iteración: 9 PINAW test:  15.3482555244 PICP test:  77.27454909819639 RMSE test:  0.676439
Iteración: 10 PINAW test:  12.6857825585 PICP test:  70.50100200400801 RMSE test:  0.578898
Ite

Iteración: 86 PINAW test:  16.5444886665 PICP test:  64.60921843687375 RMSE test:  0.891284
Iteración: 87 PINAW test:  15.0215882085 PICP test:  78.07615230460921 RMSE test:  0.680353
Iteración: 88 PINAW test:  14.3471021689 PICP test:  93.86773547094188 RMSE test:  0.46949
Iteración: 89 PINAW test:  13.7766181729 PICP test:  73.02605210420842 RMSE test:  0.638608
Iteración: 90 PINAW test:  17.274715281 PICP test:  82.3246492985972 RMSE test:  0.678033
Iteración: 91 PINAW test:  13.0375930365 PICP test:  70.86172344689379 RMSE test:  0.582528
Iteración: 92 PINAW test:  15.6750378477 PICP test:  71.14228456913828 RMSE test:  0.688295
Iteración: 93 PINAW test:  14.2779084402 PICP test:  79.19839679358718 RMSE test:  0.547273
Iteración: 94 PINAW test:  15.0590652395 PICP test:  75.55110220440882 RMSE test:  0.566874
Iteración: 95 PINAW test:  14.628551638 PICP test:  82.72545090180361 RMSE test:  0.51007
Iteración: 96 PINAW test:  14.0806208729 PICP test:  79.79959919839679 RMSE test:  0.

### 3.2 8 pasos

In [13]:
batch_size = 100
n_batches = int(np.ceil(y_train_triple.shape[0] / batch_size))
n_epochs = 8000
loss_minimo = 1000

for lam in range(600,900,5):
        
        
    loss_minimo = 1000
    grafo, X, y, outputs, init, saver, training_op, loss = red_neuronal(lam)
    with tf.Session(graph=grafo) as sess:
        sess.run(init)

        for epoch in range(n_epochs):
            for batch_indice in range(n_batches):
                X_batch, y_batch = generar_mini_batch_rnn(x_train_1, y_train_triple, batch_indice, batch_size)
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            perdida = sess.run(loss, feed_dict={X: x_test_1, y: y_test_triple})
            if perdida < loss_minimo:
                loss_minimo = perdida
                contador = 0 # reiniciar contador
            else:
                contador +=1
                if contador == 10:
                    #print('El entrenamiento ha sido detenido')
                    break 

        PINAW_percent,PICP_percent,rmse = calcular_PINAW_PICP_RNN(8,x_test,y_max,y_min,outputs)
        print('Red con parametro:' ,lam,'PINAW test: ',PINAW_percent,'PICP test: ',PICP_percent,'RMSE test: ',rmse)
        if PICP_percent > 90.0:
                print(lam)
                
                parametro = lam
                #save_path = saver.save(sess, "./my_model_intervalo1.ckpt")
                print('Ya se ha logrado el requisito de PICP > 90')
                PINAW_percent_val,PICP_percent_val,rmse_val = calcular_PINAW_PICP_RNN(8,x_val,y_max,y_min,outputs)
                print('PINAW val: ',PINAW_percent_val,'PICP val: ',PICP_percent_val,'RMSE val: ',rmse_val)

                break
                
                
                

Red con parametro: 600 PINAW test:  12.1698376522 PICP test:  68.04662379421221 RMSE test:  0.550391
Red con parametro: 605 PINAW test:  15.7862205194 PICP test:  73.63344051446946 RMSE test:  0.754681
Red con parametro: 610 PINAW test:  14.8552431409 PICP test:  78.89871382636656 RMSE test:  0.59912
Red con parametro: 615 PINAW test:  14.4382443325 PICP test:  66.59967845659163 RMSE test:  0.728919
Red con parametro: 620 PINAW test:  15.4302472558 PICP test:  77.09003215434083 RMSE test:  0.716485
Red con parametro: 625 PINAW test:  17.2903421867 PICP test:  77.81350482315112 RMSE test:  0.688799
Red con parametro: 630 PINAW test:  15.8974849808 PICP test:  87.05787781350483 RMSE test:  0.589691
Red con parametro: 635 PINAW test:  15.5248077905 PICP test:  79.30064308681672 RMSE test:  0.596722
Red con parametro: 640 PINAW test:  15.9586086024 PICP test:  80.14469453376206 RMSE test:  0.730442
Red con parametro: 645 PINAW test:  16.0237097818 PICP test:  88.38424437299035 RMSE test:  

In [14]:
best_PINAW = 100
for lam in range(0,100,1):
    loss_minimo = 1000
    grafo, X, y, outputs, init, saver, training_op, loss = red_neuronal(parametro)
    with tf.Session(graph=grafo) as sess:
        sess.run(init)
        
        for epoch in range(n_epochs):
            for batch_indice in range(n_batches):
                X_batch, y_batch = generar_mini_batch_rnn(x_train_1, y_train_triple, batch_indice, batch_size)
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            perdida = sess.run(loss, feed_dict={X: x_test_1, y: y_test_triple})
            
            if perdida < loss_minimo:
                loss_minimo = perdida
                contador = 0 # reiniciar contador
            else:
                contador +=1
                if contador == 10:
                    #print('El entrenamiento ha sido detenido')
                    break 

        PINAW_percent,PICP_percent,rmse = calcular_PINAW_PICP_RNN(8,x_test,y_max,y_min,outputs)
        print('Iteración:' ,lam,'PINAW test: ',PINAW_percent,'PICP test: ',PICP_percent,'RMSE test: ',rmse)
        if PICP_percent > 90.0 and PINAW_percent < best_PINAW:
                print(lam)
                best_PINAW = PINAW_percent
                save_path = saver.save(sess, "./mejor_modelo_rnn__LSTM_8paso.ckpt")
                print('Ya se ha logrado el requisito de PICP > 90')
                PINAW_percent_val,PICP_percent_val,rmse_val = calcular_PINAW_PICP_RNN(8,x_val,y_max,y_min,outputs)
                print('PINAW val: ',PINAW_percent_val,'PICP val: ',PICP_percent_val,'RMSE val: ',rmse_val)

Iteración: 0 PINAW test:  15.7391127946 PICP test:  86.0128617363344 RMSE test:  0.602166
Iteración: 1 PINAW test:  16.3400600309 PICP test:  81.63183279742765 RMSE test:  0.586292
Iteración: 2 PINAW test:  15.7047023787 PICP test:  81.31028938906752 RMSE test:  0.614548
Iteración: 3 PINAW test:  15.5347498689 PICP test:  81.02893890675242 RMSE test:  0.602205
Iteración: 4 PINAW test:  14.7281320165 PICP test:  86.2540192926045 RMSE test:  0.550503
Iteración: 5 PINAW test:  14.8001494244 PICP test:  78.33601286173634 RMSE test:  0.576898
Iteración: 6 PINAW test:  16.0570740454 PICP test:  77.17041800643086 RMSE test:  0.761051
Iteración: 7 PINAW test:  14.9003430164 PICP test:  78.05466237942123 RMSE test:  0.621221
Iteración: 8 PINAW test:  16.4674254284 PICP test:  93.68971061093248 RMSE test:  0.565694
8
Ya se ha logrado el requisito de PICP > 90
PINAW val:  15.6011017292 PICP val:  93.66834170854271 RMSE val:  0.567653
Iteración: 9 PINAW test:  14.8786335421 PICP test:  78.73794212

Iteración: 86 PINAW test:  16.9379232978 PICP test:  79.62218649517685 RMSE test:  0.640015
Iteración: 87 PINAW test:  14.4610617111 PICP test:  83.88263665594855 RMSE test:  0.580914
Iteración: 88 PINAW test:  15.8031641893 PICP test:  85.97266881028939 RMSE test:  0.592198
Iteración: 89 PINAW test:  16.8081568357 PICP test:  88.66559485530546 RMSE test:  0.601253
Iteración: 90 PINAW test:  16.1977252126 PICP test:  77.09003215434083 RMSE test:  0.64065
Iteración: 91 PINAW test:  14.7829433083 PICP test:  86.69614147909968 RMSE test:  0.563762
Iteración: 92 PINAW test:  15.6460234114 PICP test:  79.2604501607717 RMSE test:  0.625228
Iteración: 93 PINAW test:  15.9799039748 PICP test:  85.69131832797427 RMSE test:  0.591121
Iteración: 94 PINAW test:  15.6978642485 PICP test:  90.47427652733118 RMSE test:  0.600216
Iteración: 95 PINAW test:  15.8626713605 PICP test:  81.67202572347267 RMSE test:  0.616494
Iteración: 96 PINAW test:  17.3410188162 PICP test:  84.76688102893891 RMSE test: 

### 3.3 16 pasos

In [15]:
batch_size = 100
n_batches = int(np.ceil(y_train_triple.shape[0] / batch_size))
n_epochs = 8000
loss_minimo = 1000

for lam in range(600,900,5):
        
        
    loss_minimo = 1000
    grafo, X, y, outputs, init, saver, training_op, loss = red_neuronal(lam)
    with tf.Session(graph=grafo) as sess:
        sess.run(init)

        for epoch in range(n_epochs):
            for batch_indice in range(n_batches):
                X_batch, y_batch = generar_mini_batch_rnn(x_train_1, y_train_triple, batch_indice, batch_size)
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            perdida = sess.run(loss, feed_dict={X: x_test_1, y: y_test_triple})
            if perdida < loss_minimo:
                loss_minimo = perdida
                contador = 0 # reiniciar contador
            else:
                contador +=1
                if contador == 10:
                    #print('El entrenamiento ha sido detenido')
                    break 

        PINAW_percent,PICP_percent,rmse = calcular_PINAW_PICP_RNN(16,x_test,y_max,y_min,outputs)
        print('Red con parametro:' ,lam,'PINAW test: ',PINAW_percent,'PICP test: ',PICP_percent,'RMSE test: ',rmse)
        if PICP_percent > 90.0:
                print(lam)
                
                parametro = lam
                #save_path = saver.save(sess, "./my_model_intervalo1.ckpt")
                print('Ya se ha logrado el requisito de PICP > 90')
                PINAW_percent_val,PICP_percent_val,rmse_val = calcular_PINAW_PICP_RNN(16,x_val,y_max,y_min,outputs)
                print('PINAW val: ',PINAW_percent_val,'PICP val: ',PICP_percent_val,'RMSE val: ',rmse_val)

                break

Red con parametro: 600 PINAW test:  15.8414712877 PICP test:  72.54032258064517 RMSE test:  0.769271
Red con parametro: 605 PINAW test:  14.6169904617 PICP test:  68.62903225806451 RMSE test:  0.663876
Red con parametro: 610 PINAW test:  14.8531039454 PICP test:  84.75806451612902 RMSE test:  0.603027
Red con parametro: 615 PINAW test:  15.2148217452 PICP test:  77.74193548387098 RMSE test:  0.664637
Red con parametro: 620 PINAW test:  15.161272187 PICP test:  81.37096774193549 RMSE test:  0.607543
Red con parametro: 625 PINAW test:  16.9899853174 PICP test:  78.70967741935485 RMSE test:  0.76183
Red con parametro: 630 PINAW test:  15.3491338546 PICP test:  84.75806451612902 RMSE test:  0.598169
Red con parametro: 635 PINAW test:  15.9920310207 PICP test:  90.0 RMSE test:  0.591535
Red con parametro: 640 PINAW test:  14.9489561787 PICP test:  81.7741935483871 RMSE test:  0.610911
Red con parametro: 645 PINAW test:  16.4230913896 PICP test:  76.00806451612904 RMSE test:  0.671798
Red co

In [16]:
best_PINAW = 100
for lam in range(0,100,1):
    loss_minimo = 1000
    grafo, X, y, outputs, init, saver, training_op, loss = red_neuronal(parametro)
    with tf.Session(graph=grafo) as sess:
        sess.run(init)
        
        for epoch in range(n_epochs):
            for batch_indice in range(n_batches):
                X_batch, y_batch = generar_mini_batch_rnn(x_train_1, y_train_triple, batch_indice, batch_size)
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            perdida = sess.run(loss, feed_dict={X: x_test_1, y: y_test_triple})
            
            if perdida < loss_minimo:
                loss_minimo = perdida
                contador = 0 # reiniciar contador
            else:
                contador +=1
                if contador == 10:
                    #print('El entrenamiento ha sido detenido')
                    break 

        PINAW_percent,PICP_percent,rmse = calcular_PINAW_PICP_RNN(16,x_test,y_max,y_min,outputs)
        print('Iteración:' ,lam,'PINAW test: ',PINAW_percent,'PICP test: ',PICP_percent,'RMSE test: ',rmse)
        if PICP_percent > 90.0 and PINAW_percent < best_PINAW:
                print(lam)
                best_PINAW = PINAW_percent
                save_path = saver.save(sess, "./mejor_modelo_rnn_LSTM_16paso.ckpt")
                print('Ya se ha logrado el requisito de PICP > 90')
                PINAW_percent_val,PICP_percent_val,rmse_val = calcular_PINAW_PICP_RNN(16,x_val,y_max,y_min,outputs)
                print('PINAW val: ',PINAW_percent_val,'PICP val: ',PICP_percent_val,'RMSE val: ',rmse_val)

Iteración: 0 PINAW test:  18.1757958153 PICP test:  91.29032258064517 RMSE test:  0.611552
0
Ya se ha logrado el requisito de PICP > 90
PINAW val:  17.2403828553 PICP val:  90.91826437941474 RMSE val:  0.649651
Iteración: 1 PINAW test:  18.2394816895 PICP test:  83.10483870967742 RMSE test:  0.782006
Iteración: 2 PINAW test:  16.9345480685 PICP test:  84.15322580645162 RMSE test:  0.604556
Iteración: 3 PINAW test:  17.2980141687 PICP test:  82.01612903225806 RMSE test:  0.741032
Iteración: 4 PINAW test:  15.8638976364 PICP test:  84.83870967741936 RMSE test:  0.623495
Iteración: 5 PINAW test:  15.7990492654 PICP test:  75.20161290322581 RMSE test:  0.794084
Iteración: 6 PINAW test:  15.7433988314 PICP test:  78.75 RMSE test:  0.701021
Iteración: 7 PINAW test:  17.2024525596 PICP test:  80.24193548387096 RMSE test:  0.635045
Iteración: 8 PINAW test:  16.5156301426 PICP test:  87.13709677419355 RMSE test:  0.60249
Iteración: 9 PINAW test:  16.8129359632 PICP test:  87.66129032258064 RMSE

Iteración: 86 PINAW test:  15.9640873675 PICP test:  82.66129032258065 RMSE test:  0.633208
Iteración: 87 PINAW test:  15.6426624717 PICP test:  74.2741935483871 RMSE test:  0.706628
Iteración: 88 PINAW test:  15.3757145806 PICP test:  85.76612903225806 RMSE test:  0.577777
Iteración: 89 PINAW test:  15.348663754 PICP test:  66.00806451612902 RMSE test:  0.829518
Iteración: 90 PINAW test:  16.1552688927 PICP test:  83.62903225806451 RMSE test:  0.615165
Iteración: 91 PINAW test:  16.5841700786 PICP test:  77.58064516129032 RMSE test:  0.672612
Iteración: 92 PINAW test:  16.3283825211 PICP test:  83.26612903225806 RMSE test:  0.609472
Iteración: 93 PINAW test:  19.2813831022 PICP test:  80.68548387096774 RMSE test:  0.666333
Iteración: 94 PINAW test:  16.0404550249 PICP test:  78.50806451612902 RMSE test:  0.688708
Iteración: 95 PINAW test:  16.341097102 PICP test:  81.9758064516129 RMSE test:  0.613939
Iteración: 96 PINAW test:  15.6794523975 PICP test:  88.26612903225806 RMSE test:  0